Instructions

Get data
	Music21-corpus
		Corpus.getComposer('bach')
	Separate into voices
		Stream.elements gives you a list of different parts
	Break up into notes/rests
		Stream.flat.notesAndRests()
	Make two bigram matrices- rhythm, pitch, normalized pitch
		Rests have rhythm but pitch can just be 'Rest'
		For chords use .root()
		Normalized pitch:
			Normalize to the same key (transpose everything to one base key, say C Major/minor)
			Stream.analyze('key')
			Create interval between base key and stream key
Stream.transpose(interval)



In [6]:
duration_set = (0.125, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 6.0, 8.0, 12.0)
ds_encode_dict = {x:i for i,x in enumerate(duration_set)}
ds_decode_dict = {i:x for i,x in enumerate(duration_set)}

REST = 20

def encodeDuration(x):
    return ds_encode_dict[x]

def decodeDuration(x):
    return ds_decode_dict[x]

def encodeNote(x):
    return x - REST

def decodeNote(x):
    return x + REST
    
    

In [7]:
sorted([0.5, 1.0, 2.0, 3.0, 4.0, 6.0, 0.25, 8.0, 12.0, 0.125, 0.75, 1.5])

[0.125, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 6.0, 8.0, 12.0]

In [8]:
from music21 import corpus, converter,environment, midi, interval, pitch
from music21.stream import Stream
from music21.interval import GenericInterval
import numpy as np
us = environment.UserSettings()

try:
    us.create()
except:
    pass

us['midiPath'] = '/usr/bin/timidity'



    #for i, piece in enumerate(pieces):
    #    score = converter.parse(piece)
    #    print(piece.split("/")[-1],i,[x.partName for x in score.parts])

    #print("DONE")
    #raise ValueError()





def frequenciesFromPieces(pieces, verbose=0):


    first_note_freq = [0] * 80
    first_duration_freq = [0] * len(duration_set) 
    note_transitions = np.zeros((80,80))
    duration_transitions = np.zeros((len(duration_set),len(duration_set)))



    for i, piece in enumerate(pieces):
        if(verbose): print(piece)
        score = converter.parse(piece)
        key = score.analyze('key')
        #print(rng)
        score_interval = interval.Interval(key.tonic,pitch.Pitch('C'))
        new_score = score.transpose(score_interval)
        flattened_score = new_score.flattenParts()#["Soprano"]
        #flattened_score = new_score["Alto"]
        #flattened_score.flat.notesAndRests.stream().show('text')
        prevNote = None
        prevDuration = None
        for x in flattened_score.flat.notesAndRests.stream():
            if(x.isChord):
                note_num = x.root().midi
            else:
                note_num = x.pitch.midi
            if(x.isRest):
                note_num = REST

            e_n = encodeNote(note_num)
            e_d = encodeDuration(x.duration.quarterLength)

            if(prevNote == None):
                first_note_freq[e_n] += 1
                first_duration_freq[e_d] += 1
            else:
                note_transitions[prevNote,e_n] += 1
                duration_transitions[prevDuration,e_d] += 1

            prevNote = e_n
            prevDuration = e_d
    return first_note_freq, first_duration_freq, note_transitions, duration_transitions

def writeHDF5(name,out):
    f = h5py.File(name, 'w')
    f.create_dataset('first_note_freq', data=out[0])
    f.create_dataset('first_duration_freq', data=out[1])
    f.create_dataset('note_transitions', data=out[2])
    f.create_dataset('duration_transitions', data=out[3])
    f.close()
    
def readHDF5(name):
    f = h5py.File(name, 'r')
    out = (f["first_note_freq"][:],
           f["first_duration_freq"][:],
           f["note_transitions"][:],
           f["duration_transitions"][:])
    f.close()
    return out

def selectSimplePieces(piece):
    score = converter.parse(piece)
    return set([x.partName for x in score.parts]) == set(['Soprano', 'Alto', 'Tenor', 'Bass'])
        
        #print(first_note_priors)
        #print(first_duration_priors)
        
        #print(note_transitions)
        #print(duration_transitions)
        
        #duration_set.add(x.duration.quarterLength)
        #note_set.add(note_num)
        #print(x.isChord,x.duration.quarterLength,x.name, x.pitch.diatonicNoteNum,x.isNote)
    #print(note_set)
    #print(duration_set)
    
    #sp = midi.realtime.StreamPlayer(part)
    #sp.play()
    #print(score.flat.elements)
    #print(type(score))
    #a = score.elements.encode('utf-8').strip()
    #print(score.flat.notesAndRests())
    #break
    #for e in score.parts:
        #e = e.flat.notesAndRests()
    #    sp = midi.realtime.StreamPlayer(e)
    #    sp.play()
        
        #print(unicode(e).encode('utf8'))
    #print(score.elements[2])
    #break
    #score.flat.notesAndRests()
    #print(score)
    #score.flat.show("midi")
    #if(i > 10):
    #    break

In [ ]:
import h5py

allBach = corpus.getComposer('bach')
out = frequenciesFromPieces(allBach,verbose=1)
writeHDF5('all_bach.h5', out)

simpleBach = list(filter(selectSimplePieces, allBach))
out = frequenciesFromPieces(simpleBach,verbose=1)
writeHDF5('simple_bach.h5', out)

print(len(simpleBach),len(allBach))



/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv1.6.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv10.7.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv101.7.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv102.7.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv103.6.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv104.6.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv108.6.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv11.6.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv110.7.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv111.6.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv112.5-sc.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv112.5.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv113.8.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv114

/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv245.15.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv245.17.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv245.22.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv245.26.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv245.28.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv245.3.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv245.37.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv245.40.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv245.5.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv248.12-2.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv248.17.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv248.23-2.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv248.23-s.mxl
/usr/local/lib/python2.7/dist-packages/music21/

/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv33.6.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv330.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv331.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv332.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv333.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv334.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv335.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv336.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv337.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv338.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv339.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv340.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv341.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv342.mxl
/usr/local/lib/pyth

/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv434.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv435.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv436.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv437.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv438.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv44.7.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv45.7.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv47.5.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv48.3.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv48.7.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv5.7.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv52.6.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv55.5.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv56.5.mxl
/usr/local/l

/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv144.3.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv144.6.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv145-a.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv145.5.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv146.8.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv148.6.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv151.5.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv153.1.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv153.5.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv153.9.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv154.3.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv154.8.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv155.5.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv15

/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv282.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv283.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv284.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv285.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv286.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv287.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv288.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv289.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv290.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv291.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv292.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv293.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv294.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv295.mxl
/usr/local/lib/pytho

/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv390.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv391.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv392.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv393.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv394.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv395.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv396.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv397.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv398.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv399.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv4.8.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv40.3.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv40.6.mxl
/usr/local/lib/python2.7/dist-packages/music21/corpus/bach/bwv40.8.mxl
/usr/local/lib/py

In [ ]:
#f = h5py.File('all_bach.h5', 'r')
np.unique(readHDF5('all_bach.h5')[2])
